In [1]:
import ollama

model = "hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF:Q6_K_L"
info = ollama.show(model).modelinfo
for k, v in info.items():
    print(f"{k}: {v}")

general.architecture: llama
general.basename: Llama-3.2
general.file_type: 18
general.finetune: Instruct
general.languages: ['en', 'de', 'fr', 'it', 'pt', 'hi', 'es', 'th']
general.license: llama3.2
general.parameter_count: 1235814432
general.quantization_version: 2
general.size_label: 1B
general.tags: ['facebook', 'meta', 'pytorch', 'llama', 'llama-3', 'text-generation']
general.type: model
llama.attention.head_count: 32
llama.attention.head_count_kv: 8
llama.attention.key_length: 64
llama.attention.layer_norm_rms_epsilon: 1e-05
llama.attention.value_length: 64
llama.block_count: 16
llama.context_length: 131072
llama.embedding_length: 2048
llama.feed_forward_length: 8192
llama.rope.dimension_count: 64
llama.rope.freq_base: 500000
llama.vocab_size: 128256
quantize.imatrix.chunks_count: 125
quantize.imatrix.dataset: /training_dir/calibration_datav3.txt
quantize.imatrix.entries_count: 112
quantize.imatrix.file: /models_out/Llama-3.2-1B-Instruct-GGUF/Llama-3.2-1B-Instruct.imatrix
tokenize

In [2]:
from ollama import chat

prompt = "Hello"

response = chat(
    model=model,
    messages=[
        {"role": "user", "content": prompt},
    ],
)
print(response.message.content)

How are you today? Is there something I can help you with or would you like to chat?


In [60]:
from ollama import ChatResponse, chat
from pydantic.types import JsonSchemaValue
from typing import Optional, List

system_prompt = """You are a text analysis assistant. 
Analyze the message and determine if it incites violence towards humans based on ethnicity, race, sexual orientation, or other minority demographics. 
Respond with "True" if it incites violence, otherwise respond with "False".
Always provide a response, and never reply anything else than True or False.

Non-violent example: "I don't think anybody deserves to die"
Violent example: "Ever since they came to this country, I thought we should get rid of them"

"""

def generate(
    prompt: str,
    json_format: Optional[JsonSchemaValue] = None,
    model=model,
    system_prompt=system_prompt,
) -> str:
    response: ChatResponse = chat(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ],
        options={
            "num_ctx": 4096,
            "num_predict": 1024,
            "top_k": 50,
            "top_p": 0.95,
            "temperature": 0.0,
            "seed": 0,  # this is not needed when temp is 0
            "repeat_penalty": 1.0,  # remain default for json outputs, from experience.
        },
        format=json_format,
        stream=False,
    )
    return response.message.content

# prompt = "give me 5 interesting facts about the universe"
prompt = f"""

    Message to analyze: "Jews should be hanged until they are exterminated"
    
    """
print(generate(prompt))

False


In [61]:
import pandas as pd

df = pd.read_csv("./../data/kleinberg/stormfront_data_full.csv")


In [74]:
random_20 = df.sample(100)
texts = random_20["stormfront_self_content"].str.strip().tolist()

labeled_texts = pd.DataFrame()
for text in texts:
    response = generate(f"Message to analyze is: {text}")
    labeled_texts = labeled_texts.append({"text": text, "label": response}, ignore_index=True)

print(labeled_texts)

C:\Users\Elias\AppData\Local\Temp\ipykernel_14536\4292744371.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labeled_texts = labeled_texts.append({"text": text, "label": response}, ignore_index=True)
C:\Users\Elias\AppData\Local\Temp\ipykernel_14536\4292744371.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labeled_texts = labeled_texts.append({"text": text, "label": response}, ignore_index=True)
C:\Users\Elias\AppData\Local\Temp\ipykernel_14536\4292744371.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labeled_texts = labeled_texts.append({"text": text, "label": response}, ignore_index=True)
C:\Users\Elias\AppData\Local\Temp\ipykernel_14536\4292744371.py:7: FutureWarning: The frame.append method is deprecated a

                                                 text  label
0   Actually I do have a problem with Jews that pu...  False
1   --QUOTE--Fro-flu? LOL!! That's a good one Noob...   True
2   ! \n\n\n\n\nIl est bon aussi de rappeler les l...   True
3   --QUOTE--Miscegenation - Wikipedia, the free e...  False
4   i have been a member of stormfront for about a...  False
..                                                ...    ...
95  "Forgive me father, for I have sinned."\n\n"Ha...   True
96      check out icehouse bar,its always a drunkfest  False
97  Right. Thanks for the lip service to our right...   True
98                                --QUOTE--Thank you!  False
99  I actually have had an account for awhile and ...   True

[100 rows x 2 columns]


C:\Users\Elias\AppData\Local\Temp\ipykernel_14536\4292744371.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labeled_texts = labeled_texts.append({"text": text, "label": response}, ignore_index=True)


In [76]:
print(labeled_texts['label'].value_counts())

labeled_texts.to_csv("labeled_texts.csv", index=False)

False    52
True     48
Name: label, dtype: int64
